# Obtén datos de la API de Marvel

![imagen](https://cdn.hobbyconsolas.com/sites/navi.axelspringer.es/public/styles/hc_1440x810/public/media/image/2021/09/marvel-2460339.jpg?itok=cBnC1CBi)

Te tendrás que [registrar](https://developer.marvel.com/) y consultar la [documentación](https://developer.marvel.com/docs)

Queremos que consultes a la api para que te devuelva la info de los personajes de marvel que empiecen por la inicial de tu nombre. Deberemos guardarlos en un csv la información con la siguiente estructura

![imagen](./img/Captura_marvel.PNG)

Aquí te dejamos el código en python para poder empezar a hacer las llamadas, rellenando algunas variables como tus keys, parámetros de la llamada y la url (endpoint) a la que quieres acceder

In [1]:
import hashlib
import requests
import datetime
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 100)

In [2]:
def hash_params(timestamp,priv_key,pub_key):
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [3]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

pub_key = '7b519eca220cea84b556d6da07c75bbe'
priv_key = 'd097b014e0a40e56706de63cd61b3a954601a1de'

params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        'nameStartsWith':"M",   # Cambia este valor para la inicial de tu nombre
        #'offset':0,         # Juega con este valor
        'limit': 100            # Juega con este valor
        }

url = 'http://gateway.marvel.com/v1/public/characters'

res = requests.get(url,params=params)

In [21]:
'''
import requests
import datetime
import hashlib

def hash_params(timestamp, priv_key, pub_key):
    hash_input = f"{timestamp}{priv_key}{pub_key}"
    return hashlib.md5(hash_input.encode('utf-8')).hexdigest()

timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')
pub_key = '7b519eca220cea84b556d6da07c75bbe'
priv_key = 'd097b014e0a40e56706de63cd61b3a954601a1de'

url = 'http://gateway.marvel.com/v1/public/characters'

all_characters = {}
offset = 0
limit = 100

while True:
    params = {
        'ts': timestamp,
        'apikey': pub_key,
        'hash': hash_params(timestamp, priv_key, pub_key),
        'nameStartsWith': "M",
        'offset': offset,
        'limit': limit
    }

    print(f"Requesting with offset: {offset}")

    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        results = data['data']['results']
        
        for character in results:
            all_characters[character['id']] = character
        
        print(f"Retrieved {len(results)} results. Total accumulated: {len(all_characters)}")
        
        if len(results) < limit:  # If we get fewer results than the limit, we've reached the end
            break
        
        offset += limit  # Increase the offset by 100 for the next iteration
    else:
        print(f"Error in request: {response.status_code}")
        break

print(f"Total characters retrieved: {len(all_characters)}")
'''

Requesting with offset: 0
Retrieved 100 results. Total accumulated: 100
Requesting with offset: 100
Retrieved 57 results. Total accumulated: 157
Total characters retrieved: 157


In [4]:
res

<Response [200]>

In [22]:
#response

<Response [200]>

In [5]:
respuesta_json = res.json()
respuesta_json

{'code': 200,
 'status': 'Ok',
 'copyright': '© 2024 MARVEL',
 'attributionText': 'Data provided by Marvel. © 2024 MARVEL',
 'attributionHTML': '<a href="http://marvel.com">Data provided by Marvel. © 2024 MARVEL</a>',
 'etag': 'ceb7e5d91f494df26a9bbb14f9753be32961efb5',
 'data': {'offset': 0,
  'limit': 100,
  'total': 157,
  'count': 100,
  'results': [{'id': 1011068,
    'name': 'M (Monet St. Croix)',
    'description': '',
    'modified': '2011-09-07T09:45:28-0400',
    'thumbnail': {'path': 'http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f',
     'extension': 'jpg'},
    'resourceURI': 'http://gateway.marvel.com/v1/public/characters/1011068',
    'comics': {'available': 59,
     'collectionURI': 'http://gateway.marvel.com/v1/public/characters/1011068/comics',
     'items': [{'resourceURI': 'http://gateway.marvel.com/v1/public/comics/65072',
       'name': 'Generation X (2017) #8'},
      {'resourceURI': 'http://gateway.marvel.com/v1/public/comics/22775',
       'name': 'Ge

In [ ]:
#respuesta_json_v2 = response.json()
#respuesta_json_v2

In [6]:
import pandas as pd

# en respuesta_json
characters = respuesta_json['data']['results']

# Crear una lista de diccionarios con la información requerida
data = []
for character in characters:
    data.append({
        'id': character['id'],
        'name': character['name'],
        'picture_url': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}"
    })

# Crear el DataFrame
df = pd.DataFrame(data)

# Mostrar el DataFrame
df

,id,name,picture_url
0,1011068,M (Monet St. Croix),http://i.annihil.us/u/prod/marvel/i/mg/b/80/4c0030eabc66f.jpg
1,1011004,M.O.D.A.M.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
2,1011320,M.O.D.O.G.,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
3,1010726,M.O.D.O.K.,http://i.annihil.us/u/prod/marvel/i/mg/0/03/526034ba37851.jpg
4,1017321,M.O.D.O.K. (Iron Man 3 - The Official Game),http://i.annihil.us/u/prod/marvel/i/mg/c/00/5239c14fcddc0.jpg
5,1009418,Ma Gnuci,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
6,1009314,Mac Gargan,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
7,1010851,Mach IV,http://i.annihil.us/u/prod/marvel/i/mg/b/40/image_not_available.jpg
8,1010805,Machine Man,http://i.annihil.us/u/prod/marvel/i/mg/f/d0/4c003727804b4.jpg
9,1009411,Mad Thinker,http://i.annihil.us/u/prod/marvel/i/mg/b/c0/52740faf0d0fb.jpg


In [ ]:
'''
import pandas as pd

# en respuesta_json
characters = respuesta_json_v2['data']['results']

# Crear una lista de diccionarios con la información requerida
data = []
for character in characters:
    data.append({
        'id': character['id'],
        'name': character['name'],
        'picture_url': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}"
    })

# Crear el DataFrame
df = pd.DataFrame(data)

# Mostrar el DataFrame
df
'''

In [7]:
df.to_csv("data/marvelsuperheroestrue.csv", sep = ';', index = True)